# Azure Blob Storage

In [ ]:
from azure.mgmt.storage import StorageManagementClient
from azure.mgmt.storage.models import StorageAccountCreateParameters
from azure.storage.blob import BlobServiceClient

from git import Repo

In [ ]:
storage_client = StorageManagementClient(credential, subscription_id)

In [ ]:
from azure.mgmt.storage import StorageManagementClient
from azure.mgmt.storage.models import StorageAccountCreateParameters, SkuName
from azure.core.exceptions import HttpResponseError

def create_or_update_storage_account(storage_client, resource_group_name, account_name, location):
    try:

        storage_account = storage_client.storage_accounts.get_properties(resource_group_name, account_name)
        print(f"Storage account {account_name} already exists in '{storage_account.location}'.")
        return storage_account
    except HttpResponseError as e:
        if e.status_code == 404:

            print(f"Creating storage account {account_name}...")
            storage_parameters = StorageAccountCreateParameters(
                location=location,
                sku={'name': SkuName.standard_lrs},
                kind='StorageV2',
                enable_https_traffic_only=True
            )
            storage_operation = storage_client.storage_accounts.begin_create(
                resource_group_name,
                account_name,
                storage_parameters
            )

            storage_account = storage_operation.result()
            print(f"Storage account {account_name} created in '{location}'.")
            return storage_account
        else:
            raise
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
# Utiliza esta función con tu cliente y parámetros configurados
storage_account = create_or_update_storage_account(
    storage_client, 
    resource_group_name, 
    storage_account_name, 
    location
)

In [ ]:
def get_storage_account_keys(storage_client, resource_group_name, account_name):
    keys_response = storage_client.storage_accounts.list_keys(resource_group_name, account_name)
    keys = {key.key_name: key.value for key in keys_response.keys}
    return keys

In [ ]:
if storage_account:
    storage_keys = get_storage_account_keys(storage_client, resource_group_name, storage_account.name)
    print("Successfully retrieved the storage account keys.")
else:
    print("Failed to create or retrieve the storage account.")

Successfully retrieved the storage account keys.


In [ ]:
from azure.storage.blob import BlobServiceClient, ContainerClient

def create_blob_container(storage_account_name, storage_account_key, container_name):
    try:
        blob_service_client = BlobServiceClient(
            account_url=f"https://{storage_account_name}.blob.core.windows.net/",
            credential=storage_account_key
        )
        
        container_client = blob_service_client.get_container_client(container_name)
        container_client.create_container()
        print(f"Container '{container_name}' created.")
    except Exception as e:
        print(f"Failed to create container: {e}")



In [ ]:
storage_account_key = storage_keys['key1']

In [ ]:
create_blob_container(storage_account_name, storage_keys['key1'], container_name)

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os

def upload_files_to_blob(container_client, source_folder):
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            file_path = os.path.join(root, file)
            blob_path = os.path.relpath(file_path, start=source_folder)
            blob_client = container_client.get_blob_client(blob_path)
            
            with open(file_path, "rb") as data:
                blob_client.upload_blob(data, overwrite=True)
                print(f"Uploaded {file_path} to blob {blob_path}")

In [ ]:
blob_service_client = BlobServiceClient(
    account_url=f"https://{storage_account_name}.blob.core.windows.net/",
    credential=storage_account_key
)

container_client = blob_service_client.get_container_client(container_name)

In [ ]:
local_repo_path = '../external'

upload_files_to_blob(container_client, local_repo_path)

## Get Storage Account Keys

In [ ]:
from azure.mgmt.storage import StorageManagementClient

In [ ]:
storage_client = StorageManagementClient(credential, subscription_id)

In [ ]:
def get_storage_account_keys(storage_client, resource_group_name, account_name):
    keys_response = storage_client.storage_accounts.list_keys(resource_group_name, account_name)
    keys = {key.key_name: key.value for key in keys_response.keys}
    return keys

In [ ]:
if storage_account_name:
    storage_keys = get_storage_account_keys(storage_client, resource_group_name, storage_account_name)
    print("Successfully retrieved the storage account keys.")
else:
    print("Failed to create or retrieve the storage account.")

Successfully retrieved the storage account keys.
